## FastAI Lesson 4 - 211006957

## IMPORTS

Serão realizados os Imports necessários além de carregar o dataset escolhido

In [58]:
import os
from fastai.imports import *
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [59]:
if iskaggle:
    path = Path('../input/sms-spam-collection-dataset')
    ! pip install -q datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [60]:
!ls {path}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
spam.csv


## DATASET CONFIGURATION

In [61]:
import pandas as pd

Você irá ler o dataset que temos, este usa a codificação ISO-8859-1 que deve ser especificada no comando para que você possa lê-lo corretamente.

In [62]:
df = pd.read_csv(path/'spam.csv',encoding='ISO-8859-1')

In [63]:
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives around here though",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,"This is the 2nd time we have tried 2 contact u. U have won the å£750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate.",NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other suggestions?",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free,NaN,NaN,NaN


Algumas alterações ao dataset serão feitas, colunas que não são usadas serão removidas, v1 e v2 serão alterados por text e target, spam e não spam serão trocados por 1 e 0 respectivamente.

In [64]:
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)
df.rename(columns={'v1':'target','v2':'text'},inplace=True)
df['target']=df['target'].map({"ham":0.0,"spam":1.0})
df['input'] = 'TEXT: ' + df.text

In [65]:
df

,target,text,input
0,0.0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...","TEXT: Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,0.0,Ok lar... Joking wif u oni...,TEXT: Ok lar... Joking wif u oni...
2,1.0,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,TEXT: Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,0.0,U dun say so early hor... U c already then say...,TEXT: U dun say so early hor... U c already then say...
4,0.0,"Nah I don't think he goes to usf, he lives around here though","TEXT: Nah I don't think he goes to usf, he lives around here though"
...,...,...,...
5567,1.0,"This is the 2nd time we have tried 2 contact u. U have won the å£750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate.","TEXT: This is the 2nd time we have tried 2 contact u. U have won the å£750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate."
5568,0.0,Will Ì_ b going to esplanade fr home?,TEXT: Will Ì_ b going to esplanade fr home?
5569,0.0,"Pity, * was in mood for that. So...any other suggestions?","TEXT: Pity, * was in mood for that. So...any other suggestions?"
5570,0.0,The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free,TEXT: The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free


In [66]:
df.describe(include='object')

,text,input
count,5572,5572
unique,5169,5169
top,"Sorry, I'll call later","TEXT: Sorry, I'll call later"
freq,30,30


## TOKENIZATION

In [67]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)

In [68]:
ds

Dataset({
    features: ['target', 'text', 'input'],
    num_rows: 5572
})

In [69]:
model_nm = 'microsoft/deberta-v3-small'

O modelo pré-treinado microsoft/Deberta-v3-small será usado juntamente com a função AutoTokenizer importada da Transformers para tokenizar o texto de forma que o modelo possa usá-los.

In [70]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [71]:
def tok_func(x): return tokz(x["input"])

In [72]:
tok_ds = ds.map(tok_func, batched=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

In [73]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT: Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 [1,
  54453,
  294,
  1968,
  583,
  18350,
  49947,
  582,
  261,
  3286,
  260,
  260,
  4955,
  364,
  267,
  5554,
  1890,
  2030,
  426,
  447,
  2181,
  865,
  11709,
  260,
  260,
  260,
  33053,
  343,
  519,
  266,
  4755,
  37964,
  260,
  260,
  260,
  2])

In [74]:
tok_ds = tok_ds.rename_columns({'target':'labels'})

## TEST AND VALIDATION SETS

## Validation set

Se divide o dataset em treinamento e verificação, o de treinamento é usado para que o modelo aprenda enquanto o de validação é usado para medir o desempenho.

In [75]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4179
    })
    test: Dataset({
        features: ['labels', 'text', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1393
    })
})

## Test Set

O dataset de ensaio é criado e utilizado para verificar o funcionamento do modelo.

In [76]:
eval_df = pd.read_csv(path/'spam.csv',encoding='ISO-8859-1')
eval_df.describe()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""","MK17 92H. 450Ppw 16""","GNT:-)"""
freq,4825,30,3,2,2


In [77]:
eval_df['input'] = 'TEXT1: ' + eval_df.v1 + '; TEXT2: ' + eval_df.v2
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

## Train model

São definidos alguns dados para o funcionamento do modelo.

In [78]:
from transformers import TrainingArguments,Trainer

BS é o tamanho do lote, enquanto epochs define o número de epochs em que o modelo será treinado.

In [79]:
bs = 32
epochs = 4

lr é a taxa de aprendizagem do modelo

In [80]:
lr = 8e-5

O coeficiente Pearson é utilizado para calcular o desempenho do modelo.

In [81]:
def corr(x,y): return np.corrcoef(x,y)[0][1]

In [82]:
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [83]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [84]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [85]:
trainer.train();

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.008395,0.973509
2,No log,0.008345,0.976221
3,No log,0.006942,0.976083
4,No log,0.007335,0.976248


## PREDICTIONS

In [86]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[-0.02366638],
       [-0.02484131],
       [ 1.06738281],
       ...,
       [-0.01239777],
       [ 0.08642578],
       [ 0.06738281]])

In [89]:
preds = np.clip(preds, 0, 1)

In [90]:
preds

array([[0.        ],
       [0.        ],
       [1.        ],
       ...,
       [0.        ],
       [0.08642578],
       [0.06738281]])